# Converting matlab data into python

Use this Jupyter notebook to convert Expo data from .mat to .npy files

In [54]:
import numpy as np
import math, os
import sys
sys.path.insert(0, 'functions/'); # add this path for makeStimulus
import makeStimulus
import helper_fcns as hf
import autoreload

import pdb

# constants - directories
# base_loc = '/arc/2.2/p1/plevy/SF_diversity/sfDiv-OriModel/sfDiv-python/';
base_loc = '/users/plevy/SF_diversity/sfDiv-OriModel/sfDiv-python/';

loc_matData = 'V1/structures/'; # where are the .mat files?
loc_pyData = 'V1/structures/'; # where do you want the .npy files?

recArea = 'V1'

## The original conversion

### Converting files

First, get the .mat files to convert; then, convert unless already done 

In [55]:
files = os.listdir(loc_matData)

In [56]:
sorted(files)

['dataList.npy',
 'descrFits_poiss_sach.npy',
 'fitList_190131c_flat_chiSq.npy',
 'fitList_190131c_wght_chiSq.npy',
 'fitList_190202c_flat_chiSq.npy',
 'fitList_190202c_wght_chiSq.npy',
 'fitList_190206c_flat_chiSq.npy',
 'fitList_190206c_flat_chiSq_details.npy',
 'fitList_190206c_wght_chiSq.npy',
 'fitList_190206c_wght_chiSq_details.npy',
 'fitList_190226c_flat_chiSq.npy',
 'fitList_190226c_flat_chiSq_details.npy',
 'fitList_190226c_wght_chiSq.npy',
 'fitList_190226c_wght_chiSq_details.npy',
 'fitList_190301c_flat_chiSq.npy',
 'fitList_190301c_flat_chiSq_details.npy',
 'fitList_190301c_wght_chiSq.npy',
 'fitList_190301c_wght_chiSq_details.npy',
 'm676l1_sfm.mat',
 'm676l1_sfm.npy',
 'm676p3l13_sfm.mat',
 'm676p3l13_sfm.npy',
 'm676p3l15_sfm.mat',
 'm676p3l15_sfm.npy',
 'm676p3l6_sfm.mat',
 'm676p3l6_sfm.npy',
 'm676p3l7_sfm.mat',
 'm676p3l7_sfm.npy',
 'm678p5l06_sfm.mat',
 'm678p5l06_sfm.npy',
 'm678p5l07_sfm.mat',
 'm678p5l07_sfm.npy',
 'm678p6l11_sfm.mat',
 'm678p6l11_sfm.npy',
 'm6

In [70]:
for i in files:
#     if i.find('#') >= 0:
#         os.rename(loc_matData + i, loc_matData + i.replace('#', ''))
#         print('IGNORE: renaming %s to %s' % (loc_matData + i, loc_matData + i.replace('#', '')))
    if i.find('m6786') >=0 and i.find('.mat') >= 0: # or change to .xml/.exxd if changing names in /recordings
        r_ind = i.find('r');
        if r_ind < 0:
            r_ind = i.find('l')
            
#         rEnd_ind = i.find('#') # if changing in /recordings/
        rEnd_ind = i.find('_') # if changing in /structures/
        substr_to_replace = i[r_ind+1:rEnd_ind]
#         print('substr: %s' % substr_to_replace)
        new_str = i[0:r_ind+1] + '%02d' % int(substr_to_replace) + i[rEnd_ind:]
        if new_str == i:
            continue;
#         os.rename(loc_matData + i, loc_matData + new_str)
        print('renaming %s to %s' % (loc_matData + i, loc_matData + new_str))

In [71]:
files = os.listdir(loc_matData)

In [72]:
files = sorted(files)

In [73]:
files

['dataList.npy',
 'descrFits_poiss_sach.npy',
 'fitList_190131c_flat_chiSq.npy',
 'fitList_190131c_wght_chiSq.npy',
 'fitList_190202c_flat_chiSq.npy',
 'fitList_190202c_wght_chiSq.npy',
 'fitList_190206c_flat_chiSq.npy',
 'fitList_190206c_flat_chiSq_details.npy',
 'fitList_190206c_wght_chiSq.npy',
 'fitList_190206c_wght_chiSq_details.npy',
 'fitList_190226c_flat_chiSq.npy',
 'fitList_190226c_flat_chiSq_details.npy',
 'fitList_190226c_wght_chiSq.npy',
 'fitList_190226c_wght_chiSq_details.npy',
 'fitList_190301c_flat_chiSq.npy',
 'fitList_190301c_flat_chiSq_details.npy',
 'fitList_190301c_wght_chiSq.npy',
 'fitList_190301c_wght_chiSq_details.npy',
 'm676l01_sfm.mat',
 'm676l01_sfm.npy',
 'm676p3l06_sfm.mat',
 'm676p3l06_sfm.npy',
 'm676p3l07_sfm.mat',
 'm676p3l07_sfm.npy',
 'm676p3l13_sfm.mat',
 'm676p3l13_sfm.npy',
 'm676p3l15_sfm.mat',
 'm676p3l15_sfm.npy',
 'm678p5l06_sfm.mat',
 'm678p5l06_sfm.npy',
 'm678p5l07_sfm.mat',
 'm678p5l07_sfm.npy',
 'm678p6l11_sfm.mat',
 'm678p6l11_sfm.npy'

In [74]:
# convert individual files
unitName = [];
expType  = [];
unitArea = [];
for i in files:
    # if file has 'sfm' in it and starts with m then 
    if i.find('sfm') >= 0 and i.startswith('m67'):
        
        # don't convert if it already exists
        if os.path.exists(loc_pyData + i.replace('.mat', '.npy')):
            if i.endswith('.npy') and i.find('fullWave') == -1: # only add once (not also with .mat)
                unitName.append(i[0:i.find('_')]) # go up to the '_' character
                _, expName = hf.get_exp_ind(loc_pyData, i[0:i.find('_')])
                if expName is None:
                    pdb.set_trace();
                expType.append(expName);
                unitArea.append(recArea)
            continue;
                
        print("loading: " + i)
        matData = makeStimulus.loadmat(loc_matData + i);
        S = matData.get('S'); # the actual data structure
        _, expName = hf.get_exp_ind(loc_pyData, i[0:i.find('_')])
        
        print("now saving...")
        saveName = loc_pyData + i.replace('.mat', '.npy');
#         np.save(saveName, S)
        
        unitName.append(i[0:i.find('_')]) # go up to the '_' character
        expType.append(expName);
        unitArea.append(recArea);

In [75]:
expType

['sfMixLGN',
 'sfMixHalfInt',
 'sfMixInt',
 'sfMixHalfInt',
 'sfMixHalfInt',
 'sfMixHalfInt',
 'sfMixHalfInt',
 'sfMixHalfInt',
 'sfMixHalfInt',
 'sfMixHalfInt',
 'sfMixHalfInt',
 'sfMixHalfInt',
 'sfMixHalfInt']

In [76]:
unitName

['m676l01',
 'm676p3l06',
 'm676p3l07',
 'm676p3l13',
 'm676p3l15',
 'm678p5l06',
 'm678p5l07',
 'm678p6l11',
 'm678p6l12',
 'm678p6l15',
 'm678p6l16',
 'm678p6l18',
 'm678p7r03']

In [77]:
unitArea

['V1', 'V1', 'V1', 'V1', 'V1', 'V1', 'V1', 'V1', 'V1', 'V1', 'V1', 'V1', 'V1']

In [78]:
unitName

['m676l01',
 'm676p3l06',
 'm676p3l07',
 'm676p3l13',
 'm676p3l15',
 'm678p5l06',
 'm678p5l07',
 'm678p6l11',
 'm678p6l12',
 'm678p6l15',
 'm678p6l16',
 'm678p6l18',
 'm678p7r03']

Create/update data list

In [79]:
if os.path.exists(loc_pyData + 'dataList.npy'):
    dataList = np.load(loc_pyData + 'dataList.npy').item();
    dataList['unitName'] = unitName;
    dataList['unitArea'] = unitArea;
    dataList['expType'] = expType;
    np.save(loc_pyData + 'dataList.npy', dataList);
else: # unitType, isolation, comment must be filled in by hand at later time
    dataList = dict();
    dataList['unitName'] = unitName;
    dataList['unitArea'] = unitArea;
    dataList['expType'] = expType;
    dataList['expType'] = expType;
    dataList['isolation'] = [];
    dataList['comment'] = [];
    np.save(loc_pyData + 'dataList.npy', dataList);

Now, check the saved/updated data list

In [80]:
dataList = np.load(loc_pyData + 'dataList.npy').item();

Likely not needed, this section was from converting the previous data set where normalization responses where already computed in Matlab

In [ ]:
# change some things around...
for i in dataList['unitName']:

    print("changing: " + i)
    S = np.load(loc_pyData + i + '_sfm.npy').item(); # the actual data structure
    
    if S.get('sfm').get('mod'):
        if S.get('sfm').get('mod').get('normalization') and S.get('sfm').get('mod').get('normalization_py'):
            
            S['sfm']['mod']['norm_old'] = S['sfm']['mod']['normalization'];
            S['sfm']['mod']['normalization'] = S['sfm']['mod']['normalization_py'];
            S['sfm']['mod'].pop('normalization_py');
            
            print("now saving...")
            saveName = loc_pyData + i + '_sfm.npy';
            np.save(saveName, S)

## General 'update python structs' here

Use this section to update something about each cell in the dataList (change appropriate field(s))

In [ ]:
dL = np.load(loc_pyData + 'dataList.npy').item();

In [ ]:
for i in dL['unitName']:
    S = np.load(loc_pyData + i + '_sfm.npy').item();
    if 'descr_model' in S['sfm']:
        S['sfm'].pop('descr_model');
        np.save(loc_pyData + i + '_sfm.npy', S);
        print('Removed descr_model from ' + i);